In [ ]:
from models import models 
import pickle
import torch.nn as nn
from models import models as mdls
from torch.utils import data
from tqdm import tqdm

In [ ]:
import sys
sys.path.append("../")

In [ ]:
training_set = pickle.load("../data/augmented_data/training_loader.pkl")
testing_set = pickle.load("../data/augmented_data/testing_loader.pkl")

In [ ]:
if not isinstance(training_set, data.DataLoader):
    raise ValueError("Invalid training set type")

if not isinstance(testing_set, data.DataLoader):
    raise ValueError("Invalid testing set type")

Model Initialization

In [ ]:
from torchvision import models

loss = nn.CrossEntropyLoss()
model = mdls.FaceRecognitionNet(
    loss_function=loss,
    num_classes=4,
    learning_rate=3e-4,
    weights=models.ResNet50_Weights.DEFAULT,
    weight_decay=0.01,
)

Training Neural Network

In [ ]:
mean_loss = tqdm(model.train(image_dataset=training_set))
print('average training loss: %s' % mean_loss)

Testing model on given testing dataset

In [ ]:
predictions = model.forward(X_data=testing_set)

Evaluating testing loss using Cross-Entropy

In [ ]:
import torch 

def softmax(probs):
    return torch.sum(torch.exp(probs) / torch.sum(torch.exp(probs)))

def one_hot_encode(num_classes, y_pred):
    vec = torch.zeros(num_classes)
    vec[y_pred-1] = 1 
    return vec

def log_loss(y_pred, y_true, num_classes):
    loss = 0
    for idx, sample in enumerate(y_pred):
        soft_pred = softmax(sample)
        one_hot_vec = one_hot_encode(num_classes, y_true[idx])
        loss += one_hot_vec * torch.log2(soft_pred)
    return -loss
    

In [ ]:
test_loss = log_loss(predictions, training_set.dataset.labels)
print('loss on testing set: ', test_loss)

Saving Neural Network to ONNX format

In [ ]:
model.export(model_name='neural_net', model_path='../prod_models')

: 